In [1]:
import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt  
import os
import inspect
from tqdm import trange
#import visuals as vs
from pylab import mpl
import nltk
from nltk.corpus import stopwords

mpl.rcParams['font.sans-serif'] = ['SimHei'] # 指定默认字体
mpl.rcParams['axes.unicode_minus'] = False # 解决保存图像是负号'-'显示为方块的问题

This_File=inspect.getfile(inspect.currentframe())
Dirname=os.path.abspath(os.path.dirname(This_File))


doc_path=r"D:\HSBC\进修\Projects\毕业项目\20news-bydate\20news-bydate-train"
All_Folders=os.listdir(r'D:\HSBC\进修\Projects\毕业项目\20news-bydate\20news-bydate-train')

nltk.download('stopwords')
stop_list=list(stopwords.words('english'))

def getText(sub_doc_path,doc_name):
    txt=open(sub_doc_path+"\\"+doc_name,"rb+").read()
    txt=txt.lower()
    txt=txt.decode("utf-8","ignore")
    #txt=str(txt)
    for ch in '~!@#$%^&*()_+"-{}[]|=?./\;,<>:?':
        txt=txt.replace(ch,"")
    return txt

alphab_list=[chr(i) for i in range(97,123)]

up_alphab_list=[x.upper() for x in alphab_list]

alphab_list.extend(up_alphab_list)

Word_Count_Dict={}

Word_Overview_Dict={}

def delete_acc_to_sw(this_list,stop_words):
    new_list=[]
    for tup_inx in range(len(this_list)):
        tup=this_list[tup_inx]
        if tup[0] not in stop_words and tup[0] not in alphab_list and tup[0] not in range(10)\
            and tup[0] not in [str(i) for i in range(10)] and tup[0]!=" ":
            new_list.append(tup)
    return new_list

for folder_inx in trange(len(All_Folders)):
    folder=All_Folders[folder_inx]
    sub_doc_path=doc_path+"\\"+folder
    All_Docs=os.listdir(sub_doc_path)
    this_class_list_of_tuple=[]
    for doc in All_Docs:
        hamletTxt=getText(sub_doc_path,doc)
        words=hamletTxt.split()
        counts={}
        for word in words:
            counts[word]=counts.get(word,0)+1
        items=list(counts.items())
        items.sort(key=lambda x:x[1],reverse=True)    
        each_word_count=[i[1] for i in items]
        Word_Count=sum(each_word_count)
        Word_Count_Dict[folder+"\\"+doc]=Word_Count
        items_without_sw=delete_acc_to_sw(items,stop_list)
        items_without_sw.sort(key=lambda x:x[1],reverse=True)  
        this_class_list_of_tuple.extend(items_without_sw)

    #jj=[this_class_list_of_tuple[i][0]=="max'ax'ax'ax'ax'ax'ax'ax'ax'ax'ax'ax'ax'ax'ax'" for i in range(len(this_class_list_of_tuple))]
    this_class_list_of_tuple.sort(key=lambda x:x[1],reverse=True) 
    voca=[this_class_list_of_tuple[i][0] for i in range(len(this_class_list_of_tuple))]
    voca_freq=[this_class_list_of_tuple[i][1] for i in range(len(this_class_list_of_tuple))]
    aux_list=[voca,voca_freq]
    aux_df=pd.DataFrame(aux_list)
    aux_df=aux_df.T
    aux_df.columns=["voca","freq"]
    aux_df=aux_df.groupby("voca",as_index=False).sum()
    aux_df=aux_df.sort_values(by="freq",ascending=False)
    
    top_10_for_this_class=[(aux_df.iloc[i,0],aux_df.iloc[i,1]) for i in range(10)]
    #top_10_for_this_class=this_class_list_of_tuple[:10]
    Word_Overview_Dict[folder]=top_10_for_this_class
    
'''        
X=list(Word_Count_Dict.keys())
X.sort()
Y=[Word_Count_Dict[i] for i in X]
'''
a=pd.DataFrame(Word_Count_Dict,index = ["Count"])
a=a.T
fig = plt.figure() 
f, ax = plt.subplots(figsize=(10, 7))
ax.set_title('新闻长度散点图')  
plt.xlabel('辅助性X轴') 
plt.ylabel('单词数量')
plt.scatter(x=range(len(a)), y=a["Count"], c='r')
plt.legend('x1') 
plt.show() 
#vs.distribution(a)
###找某值对应的键
###list(Word_Count_Dict.keys())[list(Word_Count_Dict.values()).index(14)]

'''统计文章数量'''
Train_Doc_Num_Overview={}
Test_Doc_Num_Overview={}
path=r"D:\HSBC\进修\Projects\毕业项目\20news-bydate\20news-bydate-test"
classes = os.listdir(path)
    
for Class in classes:
    count = 0
    docs_path=path+"\\"+Class
    all_documents=os.listdir(docs_path)
    for document in all_documents:
        if os.path.isfile(os.path.join(docs_path,document)):
            count += 1
    Test_Doc_Num_Overview[Class]=count

train_class_list=[]
train_doc_count_list=[]
test_class_list=[]
test_doc_count_list=[]
for i,j in Train_Doc_Num_Overview.items():
    train_class_list.append(i)
    train_doc_count_list.append(j)

for i,j in Test_Doc_Num_Overview.items():
    test_class_list.append(i)
    test_doc_count_list.append(j)
    
fig2 = plt.figure() 
f1, ax1 = plt.subplots(figsize=(10, 7))
ax1.set_title('训练集各分类文章数量统计') 
plt.bar(train_class_list, train_doc_count_list)
ax1.set_xticklabels(train_class_list, rotation = 45, ha="right") 
plt.xlabel('新闻分类') 
plt.ylabel('文章数量')
plt.show()

fig3 = plt.figure() 
f2, ax2 = plt.subplots(figsize=(10, 7))
ax2.set_title('测试集各分类文章数量统计') 
plt.bar(test_class_list, test_doc_count_list,color='purple')
ax2.set_xticklabels(test_class_list, rotation = 45, ha="right") 
plt.xlabel('新闻分类') 
plt.ylabel('文章数量')
plt.show()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\win10\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:27<00:00, 10.38s/it]


<Figure size 640x480 with 0 Axes>

<Figure size 1000x700 with 1 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 1000x700 with 1 Axes>

<Figure size 640x480 with 0 Axes>

<Figure size 1000x700 with 1 Axes>